In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [2]:
# 10.2 데이터 집계

In [3]:
# 데이터 집계는 배열로부터 스칼라값을 만들어내는 모든 데이터 변환 작업을 말한다.
# 위 예제에서 mean, count, min, sum을 이용해서 스칼라값을 구했다. 
# GroupBy 객체에 대해 mean()을 수행하면 어떤 일이 생길까?

In [4]:
# [표 10-1]에 있는 것과 같은 많은 일반적인 데이터 집계는 데이터 묶음에 대한 준비된 통계를 계산해내는 최적화된 구현을 갖고 있다.
# 하지만 이 메서드만 사용해야 하는 건 아니다. 표 10-1 페이지 396-397

In [5]:
# 직접 고안한 집계함수를 사용하고 추가적으로 그룹 객체에서 이미 정의된 메서드를 연결해서 사용할 수도 있다.
# 예를 들어 quantile 메서드가 Series나 DataFrame의 컬럼의 변위치를 계산한다는 점을 생각해보자.

In [6]:
# quantile 메서드는 GroupBy만을 위해 구현되지 않았지만 Series 메서드이기 때문에 여기서 사용할 수 있다. 
# 내부적으로 GroupBy는 Series를 효과적으로 잘게 자르고 각 조각에 대해 piece.quantile(0.9)를 호출한다. 
# 그리고 이 결과들을 모두 하나의 객체로 합쳐서 반환한다. 

In [7]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [8]:
grouped = df.groupby("key1")

In [9]:
grouped["data1"].quantile(0.9)

key1
a    1.210513
b    1.713629
Name: data1, dtype: float64

In [10]:
# 자신만의 데이터 집계함수를 사용하려면 배열의 aggregate나 agg 메서드에 해당 함수를 넘기면 된다. 

In [11]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [12]:
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.598113,0.494031
b,1,2.521511,2.303410


In [13]:
# describe 같은 메서드는 데이터를 집계하지 않는데도 잘 작동함을 확인할 수 있다.

In [14]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.555881  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.705025  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.936175  1.393406   3.0  0.441920  0.283299  0.274992  0.278369   
b     1.335403  1.965781   2.0 -0.144516  1.628757 -1.296221 -0.720368   

                                    
           50%       75%       max  
key1                                
a     0.281746  0.525384  0.769023  
b    -0.144516  0.431337  1.007189  

[2 rows x 24 columns]

In [15]:
# 이에 대한 내용은 10.3절에서 더 자세히 다룬다. 

In [16]:
# NOTE_ 사용자 정의 집계함수는 일반적으로 표 10-1에 있는 함수에 비해 무척 느리게 동작하는데, 그 이유는 중간 데이터를 생성하는 과정에서 함수 호출이나 데이터 정렬 같은 오버헤드가 발생하기 때문이다.

In [17]:
# 10.2.1 컬럼에 여러 가지 함수 적용하기

In [18]:
# 앞서 살펴본 팁 데이터로 다시 돌아가자. 여기서는 read_csv 함수로 데이터를 불러온 다음 팁의 비율을 담기 위한 컬럼인 tip_pct를 추가했다.

In [19]:
tips = pd.read_csv("examples/tips.csv")

In [20]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]

In [21]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [22]:
# 이미 살펴봤듯이 Series나 DataFrame의 모든 컬럼을 집계하는 것은 mean이나 std 같은 메서드를 호출하거나 원하는 함수에 aggregate를 사용하는 것이다. 
# 하지만 컬럼에 따라 다른 함수를 사용해서 집계를 수행하거나 여러 개의 함수를 한 번에 적용하기 원한다면 이를 쉽고 간단하게 수행할 수 있다. 
# 앞으로 몇몇 예제를 통해 이를 자세히 알아볼텐데, 먼저 tips를 day와 smoker별로 묶어보자.

In [23]:
grouped = tips.groupby(["day", "smoker"])

In [24]:
# [표 10-1]의 내용과 같은 기술 통계에서는 함수 이름을 문자열로 넘기면 된다. 

In [25]:
grouped_pct = grouped["tip_pct"]

In [26]:
grouped_pct.agg("mean")

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [27]:
# 만일 함수 목록이나 함수 이름을 넘기면 함수 이름을 컬럼으로 하는 DataFrame을 얻게 된다.

In [28]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [29]:
# 여기서는 데이터 그룹에 대해 독립적으로 적용하기 위해 agg에 집계함수들의 리스트를 넘겼다. 

In [30]:
# GroupBy 객체에서 자동으로 지정하는 컬럼 이름을 그대로 쓰지 않아도 된다. 
# lambda 함수는 이름(함수 이름은 __name__ 속성으로 확인 가능하다)이 "<lambda>"인데, 이를 그대로 쓸 경우 알아보기 힘들어진다.

In [31]:
# 이때 이름과 함수가 담긴 (name, function) 튜플의 리스트를 넘기면 각 튜플에서 첫 번째 원소가 DataFrame에서 컬럼 이름으로 사용된다(2개의 튜플을 가지는 리스트가 순서대로 맵핑된다).

In [32]:
grouped_pct.agg([("foo", "mean"), ("bar", np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [33]:
# DataFrame은 컬럼마다 다른 함수를 적용하거나 여러 개의 함수를 모든 컬럼에 적용할 수 있다.
# tip_pct와 total_bill 컬럼에 대해 동일한 세 가지 통계를 계산한다고 가정하자. 

In [34]:
functions = ["count", "mean", "max"]

In [35]:
result = grouped["tip_pct", "total_bill"].agg(functions)

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_48360\3417596921.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result = grouped["tip_pct", "total_bill"].agg(functions)


In [36]:
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [37]:
# 위에서 확인할 수 있듯이 반환된 DataFrame은 계층적인 컬럼을 가지고 있으며 이는 각 컬럼을 따로 계산한 다음 concat 메서드를 이용해서 keys 인자로 컬럼 이름을 넘겨서 이어붙인 것과 동일하다.

In [38]:
result["tip_pct"]

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [39]:
# 위에서처럼 컬럼 이름과 메서드가 담긴 튜플의 리스트를 넘기는 것도 가능하다. 

In [40]:
ftuples = [("Durchschnitt", "mean"), ("Abweichung", np.var)]

In [41]:
grouped["tip_pct", "total_bill"].agg(ftuples)

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_48360\547607895.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped["tip_pct", "total_bill"].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [42]:
# 컬럼마다 다른 함수를 적용하고 싶다면 agg 메서드에 컬럼 이름에 대응하는 함수가 들어 있는 사전을 넘기면 된다.

In [43]:
grouped.agg({"tip": np.max, "size": "sum"})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [44]:
grouped.agg({"tip_pct": ["min", "max", "mean", "std"],
             "size": "sum"})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [45]:
# 단 하나의 컬럼에라도 여러 개의 함수가 적용되었다면 DataFrame은 계층적인 컬럼을 가지게 된다.

In [46]:
# 10.2.2 색인되지 않은 형태로 집계된 데이터 반환하기

In [47]:
# 지금까지 살펴본 모든 예제에서 집계된 데이터는 유일한 그룹키 조합으로 색인(어떤 경우에는 계층적 색인)되어 반환되었다. 
# 하지만 항상 이런 동작을 기대하는 것은 아니므로 groupby 메서드에 as_index=False를 넘겨서 색인되지 않도록 할 수 있다. 

In [48]:
tips.groupby(["day", "smoker"], as_index=False).mean()

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_48360\3417973839.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tips.groupby(["day", "smoker"], as_index=False).mean()


,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [49]:
# 물론 이렇게 하지 않고 색인된 결과에 대해 reset_index 메서드를 호출해서 같은 결과를 얻을 수 있다. as_index=False 옵션을 사용하면 불필요한 계산을 피할 수 있다.

In [51]:
# 10.3 Apply: 일반적인 분리-적용-병합

In [52]:
# 가장 일반적인 GroupBy 메서드의 목적은 apply인데 지금부터 다루게 될 주제다. 
# [그림 10-2]에서 볼 수 있듯이 apply 메서드는 객체를 여러 조각으로 나누고, 전달된 함수를 각 조각에 일괄 적용한 후 이를 다시 합친다.

In [53]:
# 앞서 살펴보았던 팁 데이터에서 그룹별 상위 5개의 tip_pct 값을 골라보자.
# 우선 특정 컬럼에서 가장 큰 값을 가지는 로우를 선택하는 함수를 바로 작성해보자.

In [54]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(by=column)[-n:]

In [55]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [90]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]

In [91]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525


In [56]:
# 이제 흡연자(smoker) 그룹에 대해 이 함수(top)를 apply하면 다음과 같은 결과를 얻을 수 있다.

In [57]:
tips.groupby("smoker").apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [58]:
# 위 결과를 보면 top 함수가 나뉘어진 DataFrame의 각 부분에 모두 적용이 되었고, pandas.concat을 이용해서 하나로 합쳐진 다음 그룹 이름표가 붙었다.
# 그리하여 결과는 계층적 색인을 가지고 되고 내부 색인은 원본 DataFrame의 색인값을 가지게 된다. 

In [59]:
# 만일 apply 메서드로 넘길 함수가 추가적인 인자를 받는다면 함수 이름 뒤에 붙어서 넘겨주면 된다.

In [61]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")
# 넘기는 함수 안에서 하는 일은 전적으로 코드를 짜는 사람에게 달려있다. 
# 단지 pandas 객체나 스칼라값을 반환하는 함수면 된다.

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [62]:
# 이 책 앞부분에서 GroupBy 객체에 describe 메서드를 호출했던 적이 있다.

In [63]:
result = tips.groupby("smoker")["tip_pct"].describe()

In [64]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [65]:
result.unstack("smoker")

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [66]:
# describe 같은 메서드를 호출하면 GroupBy 내부적으로 다음과 같은 단계를 수행한다.

In [67]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

In [68]:
# 10.3.1 그룹 색인 생략하기

In [69]:
# 앞서 살펴본 예제들에서 반환된 객체는 원본 객체의 각 조각에 대한 색인과 그룹 키가 계층적 색인으로 사용됨을 볼 수 있었다. 
# 이런 결과는 groupby 메서드에 group_keys=False를 넘겨서 막을 수 있다. 

In [72]:
tips.groupby("smoker", group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [73]:
# 10.3.2 변위치 분석과 버킷 분석

In [74]:
# 8장에서 본 내용을 떠올려보면 pandas의 cut과 qcut 메서드를 사용해서 선택한 크기만큼 혹은 표본 변위치에 따라 데이터를 나눌 수 있었다.
# 이 함수들을 groupby와 조합하면 데이터 묶음에 대해 변위치 분석이나 버킷 분석을 매우 쉽게 수행할 수 있다. 
# 임의의 데이터 묶음을 cut을 이용해서 등간격 구간으로 나누어보자.

In [75]:
frame = pd.DataFrame({"data1": np.random.randn(1000),
                      "data2": np.random.randn(1000)})

In [92]:
frame.head()

,data1,data2
0,1.352917,0.323886
1,0.886429,-1.683325
2,-2.001637,0.526860
3,-0.371843,1.858791
4,1.669025,-0.548419


In [93]:
quartiles = pd.cut(frame["data1"], 4)

In [94]:
quartiles.head(10)

0     (0.489, 2.208]
1     (0.489, 2.208]
2    (-2.956, -1.23]
3     (-1.23, 0.489]
4     (0.489, 2.208]
5     (-1.23, 0.489]
6     (-1.23, 0.489]
7     (-1.23, 0.489]
8     (2.208, 3.928]
9     (-1.23, 0.489]
Name: data1, dtype: category
Categories (4, interval[float64, right]): [(-2.956, -1.23] < (-1.23, 0.489] < (0.489, 2.208] < (2.208, 3.928]]

In [78]:
# cut에서 반환된 Categorical 객체는 바로 groupby로 넘길 수 있다. 
# 그러므로 data2 컬럼에 대해 몇 가지 통계를 다음과 같이 계산할 수 있다.

In [95]:
def get_stats(group):
    return pd.DataFrame(
            {"min": group.min(), "max": group.max(),
            "count": group.count(), "mean": group.mean()}
    )

In [96]:
grouped = frame.groupby(quartiles)

In [97]:
grouped.apply(get_stats)

min       max  count      mean
data1                                                     
(-2.956, -1.23] data1 -2.949343 -1.230179     95 -1.677781
                data2 -3.399312  1.670835     95 -0.057381
(-1.23, 0.489]  data1 -1.228918  0.488675    596 -0.325511
                data2 -2.989741  3.260383    596 -0.001417
(0.489, 2.208]  data1  0.489965  2.200997    298  1.063527
                data2 -3.745356  2.954439    298  0.083184
(2.208, 3.928]  data1  2.212303  3.927528     11  2.699225
                data2 -1.929776  1.765640     11  0.030607

In [98]:
grouped.agg(["min", "max", "count", "mean"])

data1                               data2                  \
                      min       max count      mean       min       max count   
data1                                                                           
(-2.956, -1.23] -2.949343 -1.230179    95 -1.677781 -3.399312  1.670835    95   
(-1.23, 0.489]  -1.228918  0.488675   596 -0.325511 -2.989741  3.260383   596   
(0.489, 2.208]   0.489965  2.200997   298  1.063527 -3.745356  2.954439   298   
(2.208, 3.928]   2.212303  3.927528    11  2.699225 -1.929776  1.765640    11   

                           
                     mean  
data1                      
(-2.956, -1.23] -0.057381  
(-1.23, 0.489]  -0.001417  
(0.489, 2.208]   0.083184  
(2.208, 3.928]   0.030607

In [83]:
# 이는 등간격 버킷이었고, 표본 변위치에 기반하여 크기가 같은 버킷을 계산하려면 qcut을 사용한다.
# 나는 labels=False를 넘겨서 변위치 숫자를 구했다.

In [85]:
grouping = pd.qcut(frame.data1, 10, labels=False)

In [86]:
grouped = frame.data2.groupby(grouping)

In [89]:
grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

min       max  count      mean
data1                                                     
(-2.956, -1.23] data1 -2.949343 -1.230179     95 -1.677781
                data2 -3.399312  1.670835     95 -0.057381
(-1.23, 0.489]  data1 -1.228918  0.488675    596 -0.325511
                data2 -2.989741  3.260383    596 -0.001417
(0.489, 2.208]  data1  0.489965  2.200997    298  1.063527
                data2 -3.745356  2.954439    298  0.083184
(2.208, 3.928]  data1  2.212303  3.927528     11  2.699225
                data2 -1.929776  1.765640     11  0.030607

In [99]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -2.949343 -0.694838    250 -1.219798
      data2 -3.399312  2.628441    250 -0.040317
1     data1 -0.689307 -0.017141    250 -0.366636
      data2 -2.630247  3.260383    250 -0.003808
2     data1 -0.017007  0.625428    250  0.302942
      data2 -3.056990  2.458842    250  0.004203
3     data1  0.627076  3.927528    250  1.256407
      data2 -3.745356  2.954439    250  0.115241

In [100]:
# pandas의 Categorical 자료형은 12장에서 자세히 살펴볼 것이다.

In [101]:
# 10.3.3 예제: 그룹에 따른 값으로 결측치 채우기

In [102]:
# 누락된 데이터를 정리할 때면 어떤 경우에는 dropna를 사용해서 데이터를 살펴보고 걸러내기도 한다.
# 하지만 어떤 경우에는 누락된 값을 고정된 값이나 혹은 데이터로부터 도출된 어떤 값으로 채우고 싶을 때도 있다.

In [103]:
# 이런 경우 fillna 메서드를 사용하는데, 누락된 값을 평균값으로 대체하는 예제를 살펴보자.

In [104]:
s = pd.Series(np.random.randn(6))

In [105]:
s[::2] = np.nan

In [106]:
s

0         NaN
1   -1.930931
2         NaN
3   -0.195258
4         NaN
5    0.422648
dtype: float64

In [107]:
s.fillna(s.mean())

0   -0.567847
1   -1.930931
2   -0.567847
3   -0.195258
4   -0.567847
5    0.422648
dtype: float64

In [109]:
# 그룹별로 채워넣고 싶은 값이 다르다고 가정해보자. 아마도 추측했듯이 데이터를 그룹으로 나누고 apply 함수를 사용해서 각 그룹에 대해 fillna를 적용하면 된다.
# 여기서 사용된 데이터는 동부와 서부로 나눈 미국의 지역에 대한 데이터다.

In [110]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]

In [111]:
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]
group_key = ["East"] * 4 + ["West"] * 4

In [112]:
data = pd.Series(np.random.randn(8), index=states)

In [113]:
data

Ohio          2.001614
New York      1.385914
Vermont      -1.104685
Florida       0.677098
Oregon        0.560161
Nevada       -0.815368
California    1.004914
Idaho         0.361801
dtype: float64

In [114]:
# ["East"] * 4는 ["East"] 리스트 안에 있는 네 벌의 원소를 이어붙인다. 
# 리스트를 더하면 각 리스트를 이어붙일 수 있다.
# 데이터에서 몇몇 값을 결측치로 만들어보자.

In [115]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan

In [116]:
data

Ohio          2.001614
New York      1.385914
Vermont            NaN
Florida       0.677098
Oregon        0.560161
Nevada             NaN
California    1.004914
Idaho              NaN
dtype: float64

In [117]:
data.groupby(group_key).mean()
data.groupby(group_key).count()
data.groupby(group_key).size()


East    4
West    4
dtype: int64

In [118]:
# 다음과 같이 누락된 값을 그룹의 평균값으로 채울 수 있다.

In [119]:
fill_mean = lambda g: g.fillna(g.mean())

In [120]:
data.groupby(group_key).apply(fill_mean)

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_48360\763740747.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data.groupby(group_key).apply(fill_mean)


Ohio          2.001614
New York      1.385914
Vermont       1.354875
Florida       0.677098
Oregon        0.560161
Nevada        0.782537
California    1.004914
Idaho         0.782537
dtype: float64

In [121]:
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_48360\1699126043.py:4: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data.groupby(group_key).apply(fill_mean)


Ohio          2.001614
New York      1.385914
Vermont       1.354875
Florida       0.677098
Oregon        0.560161
Nevada        0.782537
California    1.004914
Idaho         0.782537
dtype: float64

In [122]:
# 아니면 그룹에 따라 미리 정의된 다른 값을 채워 넣어야할 경우도 있다. 
# 각 그룹은 내부적으로 name이라는 속성을 가지고 있으므로 이를 이용하자.

In [123]:
fill_values = {"East": 0.5, "West": -1}

In [124]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [125]:
data.groupby(group_key).apply(fill_func)

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_48360\2085919508.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data.groupby(group_key).apply(fill_func)


Ohio          2.001614
New York      1.385914
Vermont       0.500000
Florida       0.677098
Oregon        0.560161
Nevada       -1.000000
California    1.004914
Idaho        -1.000000
dtype: float64

In [126]:
fill_values = {"East": 0.5, "West": -1}
def fill_func(group):
    return group.fillna(fill_values[group.name])

data.groupby(group_key).apply(fill_func)

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_48360\2633580450.py:5: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  data.groupby(group_key).apply(fill_func)


Ohio          2.001614
New York      1.385914
Vermont       0.500000
Florida       0.677098
Oregon        0.560161
Nevada       -1.000000
California    1.004914
Idaho        -1.000000
dtype: float64

In [127]:
# 10.3.4 예제: 랜덤 표본과 순열

In [128]:
# 대용량의 데이터를 몬테카를로 시뮬레이션이나 다른 애플리케이션에서 사용하기 위해 랜덤 표본을 뽑아낸다고 해보자.
# 뽑아내는 방법은 여러 가지가 있는데, 여기서 Series의 sample 메서드를 사용하자.

In [129]:
# 예시를 위해 트럼프 카드 덱을 한번 만들어보자.

In [130]:
suits = ["H", "S", "C", "D"]  # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [131]:
# 이렇게 해서 블랙잭 같은 게임에서 사용하는 카드 이름과 값을 색인으로 하는 52장의 카드가 Series 객체로 준비되었다.
# (단순히 하기 위해 A 에이스를 1로 취급했다).

In [132]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [133]:
# 이제 앞에서 얘기한 것처럼 5장의 카드를 뽑기 위해 다음 코드를 작성한다.

In [134]:
def draw(deck, n=5):
    return deck.sample(n)

In [135]:
draw(deck)

6S     6
8D     8
2C     2
QH    10
JD    10
dtype: int64

In [136]:
# 각 세트(하트, 스페이드, 클럽, 다이아몬드)별로 2장의 카드를 무작위로 뽑고 싶다고 가정하자.
# 세트는 각 카드 이름의 마지막 글자이므로 이를 이용해서 그룹을 나누고 apply를 사용한다. 

In [137]:
def get_suit(card):
    # 마지막 글자가 세트
    return card[-1]

In [138]:
deck.groupby(get_suit).apply(draw, n=2)

C  6C     6
   8C     8
D  7D     7
   QD    10
H  QH    10
   8H     8
S  KS    10
   JS    10
dtype: int64

In [139]:
# 아래와 같은 방법으로 각 세트별 2장의 카드를 무작위로 뽑을 수도 있다.

In [140]:
deck.groupby(get_suit).apply(draw, n=2)

C  9C      9
   4C      4
D  2D      2
   QD     10
H  AH      1
   10H    10
S  6S      6
   5S      5
dtype: int64

In [141]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

2C     2
3C     3
KD    10
8D     8
KH    10
3H     3
2S     2
4S     4
dtype: int64

In [142]:
# 10.3.5 예제: 그룹 가중 평균과 상관관계

In [143]:
# groupby의 나누고 적용하고 합치는 패러다임에서(그룹 가중 평균과 같은) DataFrame의 컬럼 간 연산이나 두 Series 간의 연산은 일상적이다. 
# 예를 들어 그룹 키와 값 그리고 어떤 가중치를 갖는 다음 데이터 묶음을 살펴보자.

In [144]:
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.randn(8),
                   "weights": np.random.rand(8)})

In [145]:
df

,category,data,weights
0,a,0.022100,0.779796
1,a,0.546344,0.422211
2,a,0.385143,0.962321
3,a,0.303070,0.051641
4,b,-0.346796,0.329897
5,b,-0.317939,0.576990
6,b,-0.245498,0.517064
7,b,0.004339,0.337359


In [146]:
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.standard_normal(8),
                   "weights": np.random.uniform(size=8)})

In [147]:
df

,category,data,weights
0,a,0.448674,0.331535
1,a,-1.300116,0.738078
2,a,-0.247232,0.224993
3,a,-0.548833,0.129218
4,b,0.573514,0.641297
5,b,0.277008,0.234486
6,b,0.953056,0.507843
7,b,0.978280,0.380462


In [148]:
# category별 그룹 가중 평균을 보면 다음과 같다.

In [149]:
grouped = df.groupby("category")

In [150]:
def get_wavg(group):
    return np.average(group["data"], weights=group["weights"])

In [151]:
grouped.apply(get_wavg)

category
a   -0.658354
b    0.730660
dtype: float64

In [152]:
# 좀 더 복잡한 예제로 야후! 파이낸스에서 가져온 몇몇 주식과 S&P 500 지수(종목 코드 SPX)의 종가 데이터를 살펴보자.

In [153]:
close_px = pd.read_csv("examples/stock_px.csv", parse_dates=True,
                       index_col=0)

In [154]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAPL    2214 non-null   float64
 1   MSFT    2214 non-null   float64
 2   XOM     2214 non-null   float64
 3   SPX     2214 non-null   float64
dtypes: float64(4)
memory usage: 86.5 KB


In [155]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [156]:
close_px.tail(4)

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [157]:
# 퍼센트 변화율로 일일 수익률을 계산하여 연간 SPX 지수와의 상관관계를 살펴보는 일은 흥미로울 수 있는데, 다음과 같이 구할 수 있다.
# 우선 "SPX" 컬럼과 다른 컬럼의 상관관계를 계산하는 함수를 만든다.

In [158]:
def spx_corr(group):
    return group.corrwith(group["SPX"])

In [159]:
# 그리고 pct_change 함수를 이용해서 close_px의 퍼센트 변화율을 계산한다.

In [160]:
rets = close_px.pct_change().dropna()

In [161]:
# 마지막으로 각 datetime에서 연도 속성만 반환하는 한줄 짜리 함수를 이용해서 연도별 퍼센트 변화율을 구한다.

In [162]:
def get_year(x):
    return x.year

In [163]:
by_year = rets.groupby(get_year)

In [164]:
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [165]:
# 물론 두 컬럼의 상관관계를 계산하는 것도 가능하다. 
# 다음은 애플과 마이크로소프트 주가의 연간 상관관계다.

In [166]:
def corr_aapl_msft(group):
    return group["AAPL"].corr(group["MSFT"])

In [167]:
by_year.apply(corr_aapl_msft)

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

In [168]:
# 10.3.6 예제: 그룹상의 선형회귀

In [169]:
# 이전 예제와 같은 맥락으로, pandas 객체나 스칼라값을 반환하기만 한다면 groupby를 좀 더 복잡한 그룹상의 통계 분석을 위해 사용할 수 있다.
# 예를 들어 계량경제 라이브러리인 statsmodels를 사용해서 regress라는 함수를 작성하고 각 데이터 묶음마다 최소제곱으로 회귀를 수행할 수 있다.

In [171]:
import statsmodels.api as sm

In [172]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [173]:
# 이제 SPX 수익률에 대한 애플(AAPL) 주식의 연간 선형회귀는 다음과 같이 수행할 수 있다.

In [174]:
by_year.apply(regress, "AAPL", ["SPX"])

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


In [175]:
# 10.4 피벗테이블과 교차일람표

In [176]:
# 피벗테이블은 스프레드시트 프로그램과 그 외 다른 데이터 분석 소프트웨어에서 흔히 볼 수 있는 데이터 요약화 도구다.
# 피벗테이블은 데이터를 하나 이상의 키로 수집해서 어떤 키는 로우에, 어떤 키는 컬럼에 나열해서 데이터를 정렬한다.

In [177]:
# pandas에서 피벗테이블은 이 장에서 설명했던 groupby 기능을 사용해서 계층적 색인을 활용한 재형성 연산을 가능하게 해준다.
# DataFrame에는 pivot_table 메서드가 있는데 이는 pandas 모듈의 최상위 함수로도 존재한다(pandas.pivot_table).

In [178]:
# groupby를 위한 편리한 인터페이스를 제공하기 위해 pivot_table은 마진이라고 하는 부분합을 추가할 수 있는 기능을 제공한다.

In [179]:
# 팁 데이터로 돌아가서 요일(day)과 흡연자(smoker) 집단에서 평균(pivot_table)을 구해보자.

In [180]:
tips.pivot_table(index=["day", "smoker"])

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_48360\566656882.py:1: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  tips.pivot_table(index=["day", "smoker"])


size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [181]:
# 이는 groupby를 사용해서 쉽게 구할 수 있는데, 이제 tip_pct와 size에 대해서만 집계를 하고 날짜(time)별로 그룹지어보자.
# 이를 위해 day 로우와 smoker 컬럼을 추가했다.

In [182]:
tips.pivot_table(["tip_pct", "size"], index=["time", "day"],
                 columns="smoker")

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [183]:
# 이 테이블은 margins=True를 넘겨서 부분합을 포함하도록 확장할 수 있는데, 그렇게 하면 ALL 컬럼과 ALL로우가 추가되어 단일 줄 안에서 그룹 통계를 얻을 수 있다.

In [185]:
tips.pivot_table(["tip_pct", "size"], index=["time", "day"],
                columns="smoker", margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [186]:
# 여기서 All 값은 흡연자와 비흡연자를 구분하지 않은 평균값(All 컬럼)이거나 로우에서 두 단계를 묶은 그룹의 평균값(All 로우)이다.

In [187]:
# 다른 집계함수를 사용하려면 그냥 aggfunc로 넘기면 되는데, 예를 들어 "count"나 len 함수는 그룹 크기의 교차일람표(총 개수나 빈도)를 반환한다.

In [188]:
tips.pivot_table("tip_pct", index=["time", "smoker"], columns="day",
                 aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur  All
time   smoker                             
Dinner No       3.0  45.0  57.0   1.0  106
       Yes      9.0  42.0  19.0   NaN   70
Lunch  No       1.0   NaN   NaN  44.0   45
       Yes      6.0   NaN   NaN  17.0   23
All            19.0  87.0  76.0  62.0  244

In [189]:
# 만약 어떤 조합이 비어있다면(혹은 NA 값) fill_value를 넘길 수도 있다.

In [190]:
tips.pivot_table("tip_pct", index=["time", "size", "smoker"],
                 columns="day", aggfunc="mean", fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
...                      ...       ...       ...       ...
Lunch  3    Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

[21 rows x 4 columns]

In [191]:
# [표10-2]에 pivot_table 메서드를 요약해두었다. 페이지 420

In [192]:
# 10.4.1 교차일람표

In [193]:
# 교차일람표(또는 교차표)는 그룹 빈도를 계산하기 위한 피벗테이블의 특수한 경우다.
# 다음은 위키피디아의 교차일람표 페이지에서 가져온 기본 예제다.

In [195]:
from io import StringIO
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""

In [196]:
data = pd.read_table(StringIO(data), sep="\s+")

In [197]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [198]:
# 설문 분석의 일부로서 이 데이터를 국적과 잘 쓰는 손에 따라 요약해보자.
# 이를 위해 pivot_table 메서드를 사용할 수 있지만 pandas.crosstab 함수가 훨씬 더 편리하다.

In [199]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [200]:
# crosstab 함수의 처음 두 인자는 배열이나 Series 혹은 배열의 리스트가 될 수 있다. 
# 팁 데이터에 대해 교차표를 구해보자.

In [201]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [202]:
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244

In [203]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS